# Projeto 1 - Ciência dos Dados

Nome: Beatriz Borges Zackiewicz

Nome: Renato Guedes Passarelli

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [86]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [87]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\bebec\OneDrive\Área de Trabalho\INSPER\CDADOS\CD22-2\projeto01\22-2a-cd-p1-grupo_renatogp\data


Carregando a base de dados com as notícias classificadas manualmente:

In [88]:
filename = 'dados.xlsx'

In [89]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,MT Labs,Pix agendado: você foi selecionado para ter a ...,Atenção: se o algoritmo do Google entregou ess...,13/07/2022 - 15:30,51,0
1,Economia,Gestores ‘de lado’ no câmbio com fim do ciclo ...,Uma corrida eleitoral mais acirrada do que o e...,09/08/2022 - 15:45,19,1
2,Ibovespa Futuro,Ibovespa futuro abre esta sexta-feira em leve ...,O Ibovespa futuro operava em queda na abertura...,08/07/2022 - 9:08,58,1
3,Café,Contratos futuros de café arábica ampliam perd...,Os contratos futuros do café arábica na ICE fe...,02/09/2022 - 17:32,1,0
4,Empresas,"Com prejuízo bilionário, Gol vai cortar voos; ...","Após anunciar um prejuízo de R$ 2,85 bilhões n...",30/07/2022 - 15:33,28,1


In [90]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,Mercados,"Em dia sem direção definida, Ibovespa (IBOV) f...",O Ibovespa (IBOV) fechou esta terça-feira (12)...,12/07/2022 - 17:08,53,0
1,Commodities,Soja e milho se ajustam cautelosamente esperan...,No aguardo da direção que os mercados financei...,05/08/2022 - 8:08,22,1
2,Economia,Banco Central: Previsão de alta do IPCA em 202...,Os economistas ouvidos pelo Banco Central revi...,18/07/2022 - 8:34,46,0
3,Eleições 2022,Stuhlberger vê risco de ‘república das bananas...,"Sócio da Verde Asset Management, o gestor Luis...",04/08/2022 - 7:19,24,0
4,Mercados,"Dólar reage ao Copom e inflação, mas fica bara...","O dólar comercial sobe 0,5%, perto de R$ 5,14,...",09/08/2022 - 11:43,19,1


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).
Nosso assunto escolhido é sobre o Dólar. Vamos considerar como relevates os seguintes pontos:
-Alta
-Baixa
-Queda
-Crescimento
-Mercado
-Internacional
-Estados Unidos
-Instabilidade
-Moeda
-Valor
-Preço
-Dinheiro

As irrelevantes seriam todas que não se encaixam nos padrões acima


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [100]:

from IPython.display import display
pd.options.display.max_rows = 13
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[’‘''!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed



In [105]:
from functools import total_ordering

titulo = train.Titulo.apply(cleanup) # .apply() aplica a função em cada valor da série

texto_e_categorias = train.loc[:,['Titulo','PrimeiroParag','Target']]
#print(texto_e_categorias.head(10))

#primeiro_paragraf = list(train.PrimeiroParag)
#clean_primeiro = []
#print(len(primeiro_paragraf))

tit = list(train.Titulo)
p1o = list(train.PrimeiroParag)
trgt = list(train.Target)


# ideia2 - juntar todas as palavras de cada target e ver as suas frequências
relevante_palavras = ''
irrelevante_palavras = '' 
total_vocabulario = ''

for i in range(len(train.Titulo)):
    tit_noticiai =  cleanup(str(tit[i]))
    p1o_noticiai =  cleanup(str(p1o[i]))

    tipo = trgt[i]
    if tipo == 1:
        relevante_palavras += tit_noticiai
        relevante_palavras += p1o_noticiai
    else: 
       irrelevante_palavras += p1o_noticiai
       irrelevante_palavras += tit_noticiai

    total_vocabulario += (tit_noticiai + p1o_noticiai)

total_vocabulario = pd.Series(total_vocabulario.lower().split())
pal_relevantes = pd.Series(relevante_palavras.lower().split())
pal_irrelevantes = pd.Series(irrelevante_palavras.lower().split())



In [106]:
# Frequências:
# Vocabulário total ---- procurar o jeito que tira o de/a/do/e/etc. 
total_vocabulario.value_counts()

de           866
a            428
do           392
e            386
em           383
            ... 
preocupe       1
anocompre      1
ampliaram      1
fracas         1
esfriaram      1
Length: 3598, dtype: int64

In [107]:
# Irrelevantes (Target = 0)
pal_irrelevantes.value_counts()

de                471
a                 210
do                203
e                 203
o                 187
                 ... 
agressivamente      1
apertam             1
centrais            1
bancos              1
pouco               1
Length: 2502, dtype: int64

In [108]:
# Relevantes (Target = 1)
pal_relevantes.value_counts()

de           395
a            218
em           212
do           189
e            183
            ... 
voláteis       1
fortesos       1
51578          1
030            1
esfriaram      1
Length: 1910, dtype: int64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**